In [99]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
import joblib

model = "models/svm_model.joblib"
data = pd.read_csv("./diabetes.csv")


print("Lable Balance -> ", (data["Outcome"] == 1).sum(), "|", (data["Outcome"] == 0).sum())
print()
print("Features Zero\\NaN -> \n", data.drop("Outcome", axis=1).where(data == 0).count(axis=0), sep="")

Lable Balance ->  268 | 500

Features Zero\NaN -> 
Pregnancies                 111
Glucose                       5
BloodPressure                35
SkinThickness               227
Insulin                     374
BMI                          11
DiabetesPedigreeFunction      0
Age                           0
dtype: int64


In [100]:
from sklearn.base import BaseEstimator, TransformerMixin


class ReplaceZeroWithMean(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns  # List of columns to transform

    def fit(self, X, y=None):
        self.means = {
            col: (
                int(X.loc[X[col] != 0, col].mean())
                if isinstance(X, pd.DataFrame)
                else np.where(X[:, col] != 0, X[:, col], np.nan).mean()
            )
            for col in self.columns
        }
        return self

    def transform(self, X, y=None):
        X = X.copy()
        for col in self.columns:
            if isinstance(X, pd.DataFrame):
                X.loc[X[col] == 0, col] = self.means[col]
            else:
                X[X[:, col] == 0, col] = self.means[col]
        return X

In [ ]:
X = data.drop("Outcome", axis=1)
y = data["Outcome"]

X_train, X_test, y_train, y_test = train_test_split(X, y)

steps = [
    ("replaces_zeros", ReplaceZeroWithMean(["Glucose", "BloodPressure", "SkinThickness", "BMI"])),
    ("model", joblib.load(model)),
]
pipeline = Pipeline(steps=steps)
pipeline.fit(X_train, y_train)
pipeline.score(X_test, y_test)

# pipeline.predict(X_test)

0.7552083333333334